In [1]:
import pandas as pd

In [2]:
import string

In [3]:
from math import floor

In [4]:
def parse_opcode(opcode):
    if opcode >= 100: # two-digit opcode
        op    = opcode % 100
        mode1 = floor(opcode % 1000 / 100)
        mode2 = floor(opcode % 10000 / 1000)
        mode3 = floor(opcode % 100000 / 10000)
    else:
        op = opcode
        mode1, mode2, mode3 = 0, 0, 0 # position mode
    return op, mode1, mode2, mode3

In [5]:
parse_opcode(1)

(1, 0, 0, 0)

In [6]:
parse_opcode(1002)

(2, 0, 1, 0)

In [7]:
floor(1002 % 1000 / 100)

0

In [8]:
floor(1002 % 10000 / 1000)

1

In [9]:
floor(1002 % 100000 / 10000)

0

In [10]:
def fetch_value(data, arg, mode):
    if mode == 0: # position
        return data[arg]
    elif mode == 1: # immediate
        return arg

In [11]:
fetch_value([3,0,4,0,99], 0, 0)

3

In [12]:
fetch_value([3,0,4,0,99], 42, 1)

42

In [80]:
def interpret(data, pos):
    opcode, mode1, mode2, mode3 = parse_opcode(data[pos])
    if opcode == 1: # add
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]
        
        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        assert(mode3 == 0)
        
        data[arg3] = val1 + val2
        return interpret(data, pos + 4)
    elif opcode == 2: # multiply
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]
        
        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        assert(mode3 == 0)
        
        data[arg3] = val1 * val2
        return interpret(data, pos + 4)
    elif opcode == 3: # input
        arg1 = data[pos+1]
        assert(mode1 == 0)
        data[arg1] = int(input())
        return interpret(data, pos + 2)
    elif opcode == 4: # output
        arg1 = data[pos+1]
        val1 = fetch_value(data, arg1, mode1)
        print(val1)
        return interpret(data, pos + 2)
    elif opcode == 5: # jump-if-true
        arg1 = data[pos+1]
        arg2 = data[pos+2]

        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        
        if val1 != 0:
            pos = val2
            return interpret(data, pos)
    
        return interpret(data, pos+3)
    elif opcode == 6: # jump-if-false
        arg1 = data[pos+1]
        arg2 = data[pos+2]

        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        
        if val1 == 0:
            pos = val2
            return interpret(data, pos)
        
        return interpret(data, pos+3)
    elif opcode == 7: # less than
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]

        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        assert(mode3 == 0)
        
        if val1 < val2:
            data[arg3] = 1
        else:
            data[arg3] = 0
        
        return interpret(data, pos + 4)
    elif opcode == 8: # equals
        arg1 = data[pos+1]
        arg2 = data[pos+2]
        arg3 = data[pos+3]

        val1 = fetch_value(data, arg1, mode1)
        val2 = fetch_value(data, arg2, mode2)
        assert(mode3 == 0)
        
        if val1 == val2:
            data[arg3] = 1
        else:
            data[arg3] = 0
        
        return interpret(data, pos + 4)        
    elif opcode == 99: # end
        return data

In [15]:
interpret([3,0,4,0,99], 0)

42


[42, 0, 4, 0, 99]

In [43]:
interpret([1101,100,-1,4,0], 0)

[1101, 100, -1, 4, 99]

In [58]:
interpret([3,9,8,9,10,9,4,9,99,-1,8], 0)

8
1


[3, 9, 8, 9, 10, 9, 4, 9, 99, 1, 8]

In [60]:
interpret([3,9,7,9,10,9,4,9,99,-1,8], 0)

7
1


[3, 9, 7, 9, 10, 9, 4, 9, 99, 1, 8]

In [62]:
interpret([3,3,1108,-1,8,3,4,3,99], 0)

9
0


[3, 3, 1108, 0, 8, 3, 4, 3, 99]

In [65]:
interpret([3,3,1107,-1,8,3,4,3,99], 0)

7
1


[3, 3, 1107, 1, 8, 3, 4, 3, 99]

In [84]:
interpret([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], 0)

2002
1


[3, 12, 6, 12, 15, 1, 13, 14, 13, 4, 13, 99, 2002, 1, 1, 9]

In [88]:
interpret([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], 0)

2002
1


[3, 3, 1105, 2002, 9, 1101, 0, 0, 12, 4, 12, 99, 1]

In [92]:
output_value = interpret([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], 0)

12
1001


In [72]:
def load_input():
    with open("input") as infile:
        data = infile.read()
        data = data.rstrip()
        input_data = data.split(",")
    input_data = list(map(int, input_data))
    return input_data

In [78]:
input_data = load_input()

In [79]:
output_data = interpret(input_data, 0)

1
0
0
0
0
0
0
0
0
0
15508323


In [93]:
input_data = load_input()

In [94]:
output_data = interpret(input_data, 0)

5
9006327
